In [4]:
from dataset import get_mnist_dataset
train_set, train_classes = get_mnist_dataset(train=True, shuffle=True, normalize=False)
val_set, val_classes = get_mnist_dataset(train=False, total_data=1000, shuffle=True, normalize=False)


from collections import OrderedDict, namedtuple
from itertools import product
import os
from tqdm import tqdm_notebook as tqdm
from tqdm import trange
from IPython.display import clear_output
import time
import torch.nn as nn
import pandas as pd
import torch
import matplotlib.pyplot as plt
from utils.run_manager import RunBuilder
from models import g_step, MnistAEDGCCA, MnistAutoencoderNoBN
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader
import torchvision
from dataset import get_mnist_dataset 
import numpy as np

class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

if torch.cuda.is_available():
    devices = ['cpu']
else:
    devices = ['cpu']
print('starting')


params = OrderedDict(
    lr = [0.001],
    batch_size = [1000],
    device = devices,
    shuffle = [True],
    num_workers = [5],
    manual_seed = [1265],
    loss_func = [nn.MSELoss],
    quant = [True],
    latent_dim = [10], 
    num_inner_epochs = [8]
)

# layer_sizes_list = 3*[[128, 64, 2]]
# input_size_list = 3*[2]


run_count = 0
models = []


run_data = []

data_load_time = 0
forward_time = 0


for run in RunBuilder.get_runs(params):
#     torch.cuda.set_device(run.device)
    
    run_count += 1
    device = torch.device(run.device)
    
    dgcca = MnistAEDGCCA(output_size=run.latent_dim, network=)
    dgcca = dgcca.to('cpu')
        
    train_views = list(train_set.to('cpu'))
    val_views = list(val_set.to('cpu'))
    
    optimizer = torch.optim.Adam(dgcca.parameters(), lr=run.lr)
    num_batches = len(train_views[0])//run.batch_size
    
    criterion = run.loss_func()

    out = torch.stack(dgcca(train_views))
    G = g_step(out.clone().detach())  
    
    M_serv = out.detach().clone()
    
    I = len(train_views)
    
    for epoch in range(60):
        total_recons_loss = 0
        total_val_loss = 0
        batch_count = 0
        total_dgcca_loss = 0
        
        dgcca.to('cuda')
        
        for _ in tqdm(range(run.num_inner_epochs)):
            for i in tqdm(range(num_batches)):

                optimizer.zero_grad()
                batch = []

                # mini batch gradient
                batch = [view[(i*run.batch_size):((i+1)*run.batch_size), :].to('cuda') for view in train_views]            
                target = G[(i*run.batch_size):((i+1)*run.batch_size), :].to('cuda')

                latent = dgcca(batch)

                ae_loss = (run.latent_dim/(2*28*28*target.shape[0]))*torch.norm(torch.stack(dgcca.decode(latent)) - torch.stack(batch))

                dgcca_loss = 1/2*torch.norm(torch.stack(latent)-target)/target.shape[0] 

                loss = dgcca_loss + ae_loss

                loss.backward()

                optimizer.step()

                total_recons_loss += loss.item()
                total_dgcca_loss += dgcca_loss.item()
#                 print(ae_loss.item(), dgcca_loss.item())
                del batch, target, latent

        dgcca.to('cpu')
        out = torch.stack(dgcca(train_views)).detach().clone()        
        if run.quant:
            for i in range(I):
                diff = out[i] - M_serv[i]
                max_val = diff.abs().max()
                quant = ((1/max_val)*diff[i]).round()*(max_val/1)
                var = M_serv[i] + quant
                M_serv[i] = var
                del max_val, diff, quant, var
            G = g_step(M_serv.clone().detach())
        else:
            G = g_step(out.clone().detach())   
            
        # validation loss
        out_val = dgcca(val_views)
        out_val = torch.stack(out_val)
        
        G_val = g_step(out_val.clone().detach())
        
        loss_val = criterion(out_val, G_val)
        total_val_loss += loss_val.item()
        del out, G_val, out_val

        
        results = OrderedDict()
        results['epoch'] = epoch
        results['train_loss'] = total_recons_loss/(num_batches*run.num_inner_epochs)
        results['dgcca_loss'] = total_dgcca_loss/(num_batches*run.num_inner_epochs)
        results['val_loss'] = total_val_loss
        results['batch_size'] = run.batch_size
        results['lr'] = run.lr
        if results['train_loss'] < 0.00275:
            break
        run_data.append(results)
        df3 = pd.DataFrame.from_dict(run_data, orient='columns')
        clear_output(wait=True)
#         show_latent()
        display(df3)
            
#             m.track_loss(G_adv_loss=losses['beta_kl-divergence'], G_mse_loss=losses[''], D_real_loss=total_D_real, D_fake_loss=total_D_fake, D_real_count=real_count, D_fake_count=fake_count)
#         print(epoch, "total_Gloss:",total_Gloss, "total_Dloss:",total_Dloss, "mse:",total_mse_loss, "adv: ", total_adv_loss)           
#         m.end_epoch()
        torch.save(dgcca, 'trained_models/dgcca_mnist_ae2_fed.model')

,epoch,train_loss,dgcca_loss,val_loss,batch_size,lr
0,0,0.003766,0.000414,0.000882,1000,0.001
1,1,0.003490,0.000308,0.000871,1000,0.001
2,2,0.003234,0.000321,0.000890,1000,0.001
3,3,0.003139,0.000323,0.000891,1000,0.001
4,4,0.003084,0.000323,0.000893,1000,0.001
5,5,0.003049,0.000322,0.000895,1000,0.001
6,6,0.003023,0.000322,0.000895,1000,0.001
7,7,0.003004,0.000322,0.000897,1000,0.001
8,8,0.002989,0.000321,0.000898,1000,0.001
9,9,0.002977,0.000321,0.000899,1000,0.001


In [1]:
from dataset import get_mnist_dataset
train_set, train_classes = get_mnist_dataset(train=True, shuffle=True, normalize=False)
val_set, val_classes = get_mnist_dataset(train=False, total_data=1000, shuffle=True, normalize=False)


from collections import OrderedDict, namedtuple
from itertools import product
import os
from tqdm import tqdm_notebook as tqdm
from tqdm import trange
from IPython.display import clear_output
import time
import torch.nn as nn
import pandas as pd
import torch
import matplotlib.pyplot as plt
from utils.run_manager import RunBuilder
from models import g_step, MnistAEDGCCA, MnistAELinear
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader
import torchvision
from dataset import get_mnist_dataset 
import numpy as np

class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

if torch.cuda.is_available():
    devices = ['cpu']
else:
    devices = ['cpu']
print('starting')


params = OrderedDict(
    lr = [0.001],
    batch_size = [1000],
    device = devices,
    shuffle = [True],
    num_workers = [5],
    manual_seed = [1265],
    loss_func = [nn.MSELoss],
    quant = [False],
    latent_dim = [10], 
    num_inner_epochs = [1]
)

# layer_sizes_list = 3*[[128, 64, 2]]
# input_size_list = 3*[2]


run_count = 0
models = []


run_data = []

data_load_time = 0
forward_time = 0


for run in RunBuilder.get_runs(params):
#     torch.cuda.set_device(run.device)
    
    run_count += 1
    device = torch.device(run.device)
    
    dgcca = MnistAEDGCCA(output_size=run.latent_dim, network=MnistAELinear)
    dgcca = dgcca.to('cpu')
        
    train_views = list(train_set.to('cpu'))
    val_views = list(val_set.to('cpu'))
    
    optimizer = torch.optim.Adam(dgcca.parameters(), lr=run.lr)
    num_batches = len(train_views[0])//run.batch_size
    
    criterion = run.loss_func()

    out = torch.stack(dgcca(train_views))
    G = g_step(out.clone().detach())  
    
    M_serv = out.detach().clone()
    
    I = len(train_views)
    
    for epoch in range(60):
        total_recons_loss = 0
        total_val_loss = 0
        batch_count = 0
        total_dgcca_loss = 0
        
        dgcca.to('cuda')
        
        for _ in tqdm(range(run.num_inner_epochs)):
            for i in tqdm(range(num_batches)):

                optimizer.zero_grad()
                batch = []

                # mini batch gradient
                batch = [view[(i*run.batch_size):((i+1)*run.batch_size), :].to('cuda') for view in train_views]            
                target = G[(i*run.batch_size):((i+1)*run.batch_size), :].to('cuda')

                latent = dgcca(batch)

                ae_loss = (run.latent_dim/(2*28*28*target.shape[0]))*torch.norm(torch.stack(dgcca.decode(latent)) - torch.stack(batch))

                dgcca_loss = 1/2*torch.norm(torch.stack(latent)-target)/target.shape[0] 

                loss = dgcca_loss + ae_loss

                loss.backward()

                optimizer.step()

                total_recons_loss += loss.item()
                total_dgcca_loss += dgcca_loss.item()
#                 print(ae_loss.item(), dgcca_loss.item())
                del batch, target, latent

        dgcca.to('cpu')
        out = torch.stack(dgcca(train_views)).detach().clone()        
        if run.quant:
            for i in range(I):
                diff = out[i] - M_serv[i]
                max_val = diff.abs().max()
                quant = ((1/max_val)*diff[i]).round()*(max_val/1)
                var = M_serv[i] + quant
                M_serv[i] = var
                del max_val, diff, quant, var
            G = g_step(M_serv.clone().detach())
        else:
            G = g_step(out.clone().detach())   
            
        # validation loss
        out_val = dgcca(val_views)
        out_val = torch.stack(out_val)
        
        G_val = g_step(out_val.clone().detach())
        
        loss_val = criterion(out_val, G_val)
        total_val_loss += loss_val.item()
        del out, G_val, out_val

        
        results = OrderedDict()
        results['epoch'] = epoch
        results['train_loss'] = total_recons_loss/(num_batches*run.num_inner_epochs)
        results['dgcca_loss'] = total_dgcca_loss/(num_batches*run.num_inner_epochs)
        results['val_loss'] = total_val_loss
        results['batch_size'] = run.batch_size
        results['lr'] = run.lr
        if results['train_loss'] < 0.00290:
            break
        run_data.append(results)
        df4 = pd.DataFrame.from_dict(run_data, orient='columns')
        clear_output(wait=True)
#         show_latent()
        display(df4)
            
#             m.track_loss(G_adv_loss=losses['beta_kl-divergence'], G_mse_loss=losses[''], D_real_loss=total_D_real, D_fake_loss=total_D_fake, D_real_count=real_count, D_fake_count=fake_count)
#         print(epoch, "total_Gloss:",total_Gloss, "total_Dloss:",total_Dloss, "mse:",total_mse_loss, "adv: ", total_adv_loss)           
#         m.end_epoch()
        torch.save(dgcca, 'trained_models/dgcca_mnist_linear_innerit=1.model')

/scratch/sagar/Projects/learn/compressai/venv/lib/python3.6/site-packages/torch/nn/functional.py:3888: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "
/scratch/sagar/Projects/learn/compressai/venv/lib/python3.6/site-packages/torch/nn/functional.py:3826: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "
/scratch/sagar/Projects/learn/compressai/venv/lib/python3.6/site-packages/torch/nn/functional.py:3826: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify al

starting


/scratch/sagar/Projects/learn/compressai/venv/lib/python3.6/site-packages/ipykernel_launcher.py:103: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

/scratch/sagar/Projects/learn/compressai/venv/lib/python3.6/site-packages/ipykernel_launcher.py:104: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/60 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (784) must match the size of tensor b (28) at non-singleton dimension 4

In [8]:
train_set[0]

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

In [ ]:
train_set

In [7]:
6/32

0.1875